# Parse, Chunk and Load Documents 

The following notebook executes three steps: 
- **Parsing and Chunking**: The first part of the notebook parses and chunks the documents.  This is done by the [PyPDFLoader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pypdf) of LangChain. More documentation can be found here: [LangChain API](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html). 
- **Embeddings**: For every chunk an embeddings is created. For these an OpenAI Embeddings model is used: [text-embedding-3-small](https://platform.openai.com/docs/models/embeddings). 
- **Load to Database**: The Documents and Chunks are loaded to Neo4j. This is done using the [Python Driver](https://neo4j.com/docs/api/python-driver/current/) that enables querying from a Python script.

In [1]:
%pip install pypdf langchain_community langchain langchain_openai IPython neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
from neo4j import Query, GraphDatabase, RoutingControl, Result
import ast
from IPython.display import clear_output

## Get Credentials

In [3]:
env_file = 'credentials.env'

In [4]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

In [5]:
documents_path = "documents/"

## Parse and Chunk Documents

In [6]:
chunk_size = 1000
chunk_overlap = 100

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    is_separator_regex = False,
)

In [8]:
directory = os.fsencode(documents_path)
chunk_seq_id = 0
chunks_with_metadata = []

for doc in os.listdir(directory):
    doc_name = os.fsdecode(doc)
    print(f"Parsing: {doc_name}")
    doc_path = documents_path + doc_name
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    num_chunks = 0
    for page in pages:
        chunks = text_splitter.split_text(page.page_content)
        for chunk in chunks:
            d = {
                'file': page.metadata['source'],
                'page': page.metadata['page'],
                'chunks': chunk,
                'num_chuncks': len(chunks),
                'chunk_seq_id': chunk_seq_id
            }
            chunk_seq_id += 1
            num_chunks += 1
            chunks_with_metadata.append(d.copy())
    print(f"chunked {len(pages)} pages in {num_chunks} chunks")

Parsing: Interpolis Short-Term Travel Insurance.pdf
chunked 22 pages in 46 chunks
Parsing: Rabo SpaarRekening 2020.pdf
chunked 14 pages in 44 chunks
Parsing: Terms & Conditions for Online Business Services - April 2024.pdf
chunked 29 pages in 88 chunks
Parsing: Payment and Online Services Terms Sept 2022.pdf
chunked 112 pages in 354 chunks
Parsing: Rabo Beleggersrekening Terms 2020.pdf
chunked 25 pages in 78 chunks


Create a DataFrame of Chunks

In [9]:
df = pd.DataFrame.from_dict(chunks_with_metadata)

In [10]:
df

,file,page,chunks,num_chuncks,chunk_seq_id
0,documents/Interpolis Short-Term Travel Insuran...,0,Interpolis\nKortlopende\nReisverzekering\nVerz...,1,0
1,documents/Interpolis Short-Term Travel Insuran...,1,2 van 22 Verzekeringsvoorwaarden Interpolis Ko...,4,1
2,documents/Interpolis Short-Term Travel Insuran...,1,3. Welke gebeurtenissen zijn verzekerd? En wel...,4,2
3,documents/Interpolis Short-Term Travel Insuran...,1,5. Hoe wordt de hoogte van de schade vastgeste...,4,3
4,documents/Interpolis Short-Term Travel Insuran...,1,6.2 Wat als verzekerde zich daar niet aan houd...,4,4
...,...,...,...,...,...
605,documents/Rabo Beleggersrekening Terms 2020.pdf,23,"Pagina 23/24\nAlgemene voorwaarden 2020, Rabo ...",4,605
606,documents/Rabo Beleggersrekening Terms 2020.pdf,23,ekeninghouder worden geheel of gedeeltelijk ov...,4,606
607,documents/Rabo Beleggersrekening Terms 2020.pdf,23,veneens van toepassing als zich vergelijkbare ...,4,607
608,documents/Rabo Beleggersrekening Terms 2020.pdf,23,17. Geen w\nettelijk herroepingsrecht\n Bij s...,4,608


## Create embeddings

Load an embedding model

In [11]:
embeddings_model = OpenAIEmbeddings(
    model = EMBEDDINGS_MODEL,
    openai_api_key = OPENAI_API_KEY
)

Add an embedding for every chunk in the DataFrame

In [12]:
df['embedding'] = df['chunks'].apply(lambda x: embeddings_model.embed_query(x))

## Create Neo4j Connection

Setup the Python Driver for Neo4j with the loaded credentials

In [13]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the Connection

In [14]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,0


## Load to Database

Create some constraints

In [15]:
driver.execute_query(
    'CREATE CONSTRAINT unique_document IF NOT EXISTS FOR (d:Document) REQUIRE d.id IS UNIQUE',
    database_=DATABASE,
    routing_=RoutingControl.WRITE
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x177f36a10>, keys=[])

In [16]:
driver.execute_query(
    'CREATE CONSTRAINT unique_chunk IF NOT EXISTS FOR (c:Chunk) REQUIRE c.id IS UNIQUE',
    database_=DATABASE,
    routing_=RoutingControl.WRITE
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x310435c50>, keys=[])

In [17]:
schema_result_df  = driver.execute_query(
    'SHOW CONSTRAINTS',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
schema_result_df.head()

,id,name,type,entityType,labelsOrTypes,properties,ownedIndex,propertyType
0,5,unique_chunk,UNIQUENESS,NODE,[Chunk],[id],unique_chunk,None
1,3,unique_document,UNIQUENESS,NODE,[Document],[id],unique_document,None


### Load Documents Nodes to database

Create Dataframe from the documents

In [18]:
document_df = df['file'].drop_duplicates().copy()
document_df = document_df.reset_index().drop('index',axis=1).reset_index()
document_df = document_df.rename(columns={"index": "doc_id", "file": "file_location"})
document_df['file_name'] = document_df['file_location'].apply(lambda x: x.split('/')[-1])
document_df

,doc_id,file_location,file_name
0,0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
1,1,documents/Rabo SpaarRekening 2020.pdf,Rabo SpaarRekening 2020.pdf
2,2,documents/Terms & Conditions for Online Busine...,Terms & Conditions for Online Business Service...
3,3,documents/Payment and Online Services Terms Se...,Payment and Online Services Terms Sept 2022.pdf
4,4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf


Get number of pages per file

In [19]:
df = pd.merge(df, document_df, left_on='file', right_on='file_location', how='left').copy()

In [20]:
df

,file,page,chunks,num_chuncks,chunk_seq_id,embedding,doc_id,file_location,file_name
0,documents/Interpolis Short-Term Travel Insuran...,0,Interpolis\nKortlopende\nReisverzekering\nVerz...,1,0,"[0.002485640812665224, -0.009058780036866665, ...",0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
1,documents/Interpolis Short-Term Travel Insuran...,1,2 van 22 Verzekeringsvoorwaarden Interpolis Ko...,4,1,"[0.008162065409123898, -0.012821408919990063, ...",0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
2,documents/Interpolis Short-Term Travel Insuran...,1,3. Welke gebeurtenissen zijn verzekerd? En wel...,4,2,"[0.020432598888874054, -0.030086711049079895, ...",0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
3,documents/Interpolis Short-Term Travel Insuran...,1,5. Hoe wordt de hoogte van de schade vastgeste...,4,3,"[0.01846371963620186, -0.01696145534515381, 0....",0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
4,documents/Interpolis Short-Term Travel Insuran...,1,6.2 Wat als verzekerde zich daar niet aan houd...,4,4,"[0.014788197353482246, -0.03197867050766945, 0...",0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf
...,...,...,...,...,...,...,...,...,...
605,documents/Rabo Beleggersrekening Terms 2020.pdf,23,"Pagina 23/24\nAlgemene voorwaarden 2020, Rabo ...",4,605,"[-0.030459202826023102, -0.025620141997933388,...",4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf
606,documents/Rabo Beleggersrekening Terms 2020.pdf,23,ekeninghouder worden geheel of gedeeltelijk ov...,4,606,"[-0.010802646167576313, -0.016468940302729607,...",4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf
607,documents/Rabo Beleggersrekening Terms 2020.pdf,23,veneens van toepassing als zich vergelijkbare ...,4,607,"[-0.014722777530550957, -0.007933342829346657,...",4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf
608,documents/Rabo Beleggersrekening Terms 2020.pdf,23,17. Geen w\nettelijk herroepingsrecht\n Bij s...,4,608,"[-0.018603336066007614, -0.03800157457590103, ...",4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf


In [21]:
pages_df = df.groupby(['doc_id', 'file_name']).max(['page'])['page'].apply(lambda x: x+1)

In [22]:
document_df = pd.merge(document_df, pages_df, on='doc_id', how='left')
document_df

,doc_id,file_location,file_name,page
0,0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf,22
1,1,documents/Rabo SpaarRekening 2020.pdf,Rabo SpaarRekening 2020.pdf,14
2,2,documents/Terms & Conditions for Online Busine...,Terms & Conditions for Online Business Service...,29
3,3,documents/Payment and Online Services Terms Se...,Payment and Online Services Terms Sept 2022.pdf,80
4,4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf,25


### Load the Documents

In [23]:
merge_file_query = """
    MERGE(mergedDocument:Document {id: $doc_id})
    SET mergedDocument.file_location = $file_location,
        mergedDocument.file_name = $file_name,
        mergedDocument.pages = $file_pages
    RETURN mergedDocument
    """

In [24]:
document_df

,doc_id,file_location,file_name,page
0,0,documents/Interpolis Short-Term Travel Insuran...,Interpolis Short-Term Travel Insurance.pdf,22
1,1,documents/Rabo SpaarRekening 2020.pdf,Rabo SpaarRekening 2020.pdf,14
2,2,documents/Terms & Conditions for Online Busine...,Terms & Conditions for Online Business Service...,29
3,3,documents/Payment and Online Services Terms Se...,Payment and Online Services Terms Sept 2022.pdf,80
4,4,documents/Rabo Beleggersrekening Terms 2020.pdf,Rabo Beleggersrekening Terms 2020.pdf,25


In [25]:
for index, row in document_df.iterrows():
    print(row)
    clear_output(wait=True)
    driver.execute_query(
        merge_file_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        doc_id = row.doc_id,
        file_location = row.file_location,
        file_name = row.file_name,
        file_pages = row.page
    )
    print(f"Loaded {row['file_name']}")
    print("Progress: ", np.round((index+1)/document_df.shape[0]*100,2), "%")

Loaded Rabo Beleggersrekening Terms 2020.pdf
Progress:  100.0 %


### Load Chunk Nodes to database

Create Dataframe for chunks

In [26]:
chunks_df = df[['chunk_seq_id', 'num_chuncks', 'page', 'chunks', 'embedding']]
chunks_df

,chunk_seq_id,num_chuncks,page,chunks,embedding
0,0,1,0,Interpolis\nKortlopende\nReisverzekering\nVerz...,"[0.002485640812665224, -0.009058780036866665, ..."
1,1,4,1,2 van 22 Verzekeringsvoorwaarden Interpolis Ko...,"[0.008162065409123898, -0.012821408919990063, ..."
2,2,4,1,3. Welke gebeurtenissen zijn verzekerd? En wel...,"[0.020432598888874054, -0.030086711049079895, ..."
3,3,4,1,5. Hoe wordt de hoogte van de schade vastgeste...,"[0.01846371963620186, -0.01696145534515381, 0...."
4,4,4,1,6.2 Wat als verzekerde zich daar niet aan houd...,"[0.014788197353482246, -0.03197867050766945, 0..."
...,...,...,...,...,...
605,605,4,23,"Pagina 23/24\nAlgemene voorwaarden 2020, Rabo ...","[-0.030459202826023102, -0.025620141997933388,..."
606,606,4,23,ekeninghouder worden geheel of gedeeltelijk ov...,"[-0.010802646167576313, -0.016468940302729607,..."
607,607,4,23,veneens van toepassing als zich vergelijkbare ...,"[-0.014722777530550957, -0.007933342829346657,..."
608,608,4,23,17. Geen w\nettelijk herroepingsrecht\n Bij s...,"[-0.018603336066007614, -0.03800157457590103, ..."


In [27]:
merge_chunck_query = """
    MERGE(mergedChunk:Chunk {id: $chunk_seq_id})
        ON CREATE SET
            mergedChunk.page = $page,
            mergedChunk.chunk = $chunk,
            mergedChunk.embedding = $embedding
    RETURN mergedChunk
"""

In [28]:
for index, row in chunks_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_chunck_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        chunk_seq_id = row.chunk_seq_id,
        page = row.page,
        chunk = row.chunks,
        embedding = row.embedding
    )
    print("Progress: ", np.round(((index+1)/chunks_df.shape[0])*100,2), "%")

Progress:  100.0 %


### Load File to Chunk Relationship

In [29]:
part_of_df = df[['chunk_seq_id', 'doc_id']].copy()
part_of_df

,chunk_seq_id,doc_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
605,605,4
606,606,4
607,607,4
608,608,4


In [30]:
merge_part_of_query = """
    MATCH
        (doc:Document {id: $doc_id}),
        (chunk:Chunk {id: $chunk_id})
    MERGE (doc)<-[r:PART_OF]-(chunk)
    RETURN doc.name, type(r), chunk.title
"""

In [31]:
for index, row in part_of_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_part_of_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        doc_id = row.doc_id,
        chunk_id = row.chunk_seq_id
    )
    # print(f"Loaded relationship from document {row['doc_id']} to chunk {row['chunk_seq_id']}")
    print("Progress: ", np.round(((index+1)/part_of_df.shape[0])*100,2), "%")

Progress:  100.0 %


## Load Chunk to Chunk Relationship

Link the chunks in order by the "NEXT" relationship.

In [32]:
next_query = """
    MATCH (doc:Document)
    WITH doc
    CALL (doc) {
        MATCH (doc)<-[:PART_OF]-(chunks:Chunk)
        WITH chunks ORDER BY chunks.id ASC
        WITH collect(chunks) as chunk_list
        CALL apoc.nodes.link(
            chunk_list,
            "NEXT",
            {avoidDuplicates: true}
        )
        RETURN size(chunk_list) as size_chunk_list
    }
    WITH doc, size_chunk_list
    RETURN doc, size_chunk_list
"""

In [33]:
 driver.execute_query(
        next_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

EagerResult(records=[<Record doc=<Node element_id='4:c6d37cd5-dbfb-40c4-9e15-be5560ce9c92:906' labels=frozenset({'Document'}) properties={'file_location': 'documents/Interpolis Short-Term Travel Insurance.pdf', 'pages': 22, 'file_name': 'Interpolis Short-Term Travel Insurance.pdf', 'id': 0}> size_chunk_list=46>, <Record doc=<Node element_id='4:c6d37cd5-dbfb-40c4-9e15-be5560ce9c92:907' labels=frozenset({'Document'}) properties={'file_location': 'documents/Rabo SpaarRekening 2020.pdf', 'pages': 14, 'file_name': 'Rabo SpaarRekening 2020.pdf', 'id': 1}> size_chunk_list=44>, <Record doc=<Node element_id='4:c6d37cd5-dbfb-40c4-9e15-be5560ce9c92:908' labels=frozenset({'Document'}) properties={'file_location': 'documents/Terms & Conditions for Online Business Services - April 2024.pdf', 'pages': 29, 'file_name': 'Terms & Conditions for Online Business Services - April 2024.pdf', 'id': 2}> size_chunk_list=88>, <Record doc=<Node element_id='4:c6d37cd5-dbfb-40c4-9e15-be5560ce9c92:909' labels=froze

### Create Product Types

Create products associated to the documents

In [62]:
product_query = """
    MERGE (spaar:ProductType {name: 'SpaarRekening'})
    MERGE (direct:ProductType {name: 'DirectRekening'})
    MERGE (reis:ProductType {name: 'Kortlopende Reis'})
    MERGE (beleggers:ProductType {name: 'BeleggersRekening'})
    MERGE (rbb:ProductType {name: 'RaboBusiness Banking'})

    WITH spaar, direct, reis, beleggers, rbb
    SET spaar.id = 0
    SET direct.id = 1
    SET reis.id = 2
    SET beleggers.id = 3
    SET rbb.id = 4
    
    WITH spaar, direct, reis, beleggers, rbb
    MATCH (doc1:Document {file_name: 'Rabo SpaarRekening 2020.pdf'})
    MATCH (doc2:Document {file_name: 'Payment and Online Services Terms Sept 2022.pdf'})
    MATCH (doc3:Document {file_name: 'Interpolis Short-Term Travel Insurance.pdf'})
    MATCH (doc4:Document {file_name: 'Rabo Beleggersrekening Terms 2020.pdf'})
    MATCH (doc5:Document {file_name: 'Terms & Conditions for Online Business Services - April 2024.pdf'})

    MERGE (doc1)-[:RELATED_TO]->(spaar)
    MERGE (doc2)-[:RELATED_TO]->(direct)
    MERGE (doc3)-[:RELATED_TO]->(reis)
    MERGE (doc4)-[:RELATED_TO]->(beleggers)
    MERGE (doc5)-[:RELATED_TO]->(rbb)
"""

In [63]:
driver.execute_query(
        product_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

[#E5D9]  _: <CONNECTION> error: Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))): BrokenPipeError(32, 'Broken pipe')
Transaction failed and will be retried in 0.968291487689229s (Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x3151e85d0>, keys=[])

### Create Customers

Create some fake customers in the database

In [64]:
customer_query = """
WITH [
  'Jan', 'Piet', 'Klaas', 'Henk', 'Bram', 'Sven', 'Lars', 'Tom', 'Rik', 'Daan',
  'Sophie', 'Emma', 'Julia', 'Tess', 'Lisa', 'Anna', 'Sara', 'Noa', 'Lotte', 'Eva'
] AS firstNames,
[
  'Jansen', 'De Vries', 'Van den Berg', 'Bakker', 'Visser', 'Smit', 'Meijer', 'Mulder', 'Bos', 'Vos'
] AS lastNames

UNWIND range(1, 50) AS i
CALL (i, firstNames, lastNames){
    WITH i,
       apoc.coll.randomItem(firstNames) AS firstName,
       apoc.coll.randomItem(lastNames) AS lastName
  WITH i, firstName, lastName, firstName + ' ' + lastName AS fullName
  CREATE (c:Customer {
    id: i,
    firstName: firstName,
    lastName: lastName,
    name: fullName
  })
}
"""

In [65]:
driver.execute_query(
        customer_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x3151a5650>, keys=[])

### Link Customers to Products

Randomly link customers to products

In [68]:
products_query = """
    MATCH (c:Customer)
    // Randomly assign 1 to 3 product types
    WITH c, [
    {name: 'SpaarRekening'},
    {name: 'DirectRekening'},
    {name: 'Kortlopende Reis'},
    {name: 'BeleggersRekening'},
    {name: 'RaboBusiness Banking'}
    ] AS productTypes
    WITH c, apoc.coll.shuffle(productTypes)[..toInteger(rand()*3)+1] AS selectedTypes
    UNWIND selectedTypes AS pt
    MATCH (ptNode:ProductType {name: pt.name})
    
    // Generate product ID based on type
    CALL (pt){
    RETURN 
      CASE pt.name
        WHEN 'SpaarRekening', 'DirectRekening'
          THEN 'NL' + apoc.text.random(2, "0-9") + 'RABO' + apoc.text.random(10, "0-9")
        ELSE randomUUID()
      END AS productId
    }
    
    CREATE (p:Product {id: productId, name: pt.name + ' Product'})
    MERGE (c)-[:HAS_PRODUCT]->(p)
    MERGE (p)-[:OF_TYPE]->(ptNode)
    RETURN count(*) AS created
"""

In [69]:
driver.execute_query(
        products_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

EagerResult(records=[<Record created=104>], summary=<neo4j._work.summary.ResultSummary object at 0x31524d9d0>, keys=['created'])